<a href="https://colab.research.google.com/github/noobylub/Computational-Linguistic/blob/master/ComputingPairwise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem setting

In the class, we faced the problem of computing all pairwise distances between rows of matrices A and B. We used a loop over the rows of one of the matrices to save time, but this is terribly slow because it cannot be parallelised and because Python loops are slow in general. We can do better.

Below are two ways of doing this in a matrix-oriented way. The first way relies on first copying the data and then working with now-aligned pairs of rows. This wastes memory and time on data copying, but the upside is that when the rows are correctly aligned, you can then apply any function to them, not necessarily the Euclidean distance.

The second option is much cleaner, but it relies on the definition of Euclidean distance. You can compute pairwise cosine similarities in a similar (and even simpler) way, but other distance measures, e.g. Manhattan distance, will only work with the first approach.

## Option 1: Copy, then compute

If A has m rows and B has n rows, we need to construct m-times-n pairs of rows. This amounts to copying A n times, B m times and rearranging the rows.

In [1]:
import numpy as np

m = 13
n = 11
c = 20

A = np.random.rand(m, c)
B = np.random.rand(n, c)

In [2]:
A_indices = list(range(A.shape[0]))   # 0, 1, 2, 3, ..., m-1
# n copies of the first row of A, then n copies of the second row of A, etc.
# We're copying a list and then sorting -- the idiomatic numpy for this is `repeat`,
# which will produce n copies of the first element, followed by n copies of the
# second element, etc.
A_indices = sorted(A_indices * n)     # 0, 0, 0, ..., m-1, m-1
A_aligned = A[A_indices]

# n copies of each row of A should map to n rows of B in the original order
# We're copying a list without sorting -- the idiomatic numpy for this is `tile`,
# which will copy the whole array along a new axis.
B_indices = list(range(B.shape[0]))
B_aligned = B[B_indices * m]

# This will give a vector of length m*n, which we convert into a distance
# matrix
distances_1 = np.linalg.norm(A_aligned - B_aligned, axis=1).reshape(m, n)

This was better than looping, but actually we can use broadcasting behaviour of Numpy to express this more succinctly.

In [3]:
A_aligned = A.reshape(m, 1, c)  # or A[:, None, :]
B_aligned = B.reshape(1, n, c)  # or B[None, :, :]

# In order to obtain a matching shape, numpy will broadcast B_aligned
# along the second dimension and A_aligned along the first dimension.
# Now, however, we a have 3d array of shape (m, n, c), so we need to
# take the norm of the difference along the third axis.
distances_2 = np.linalg.norm(A_aligned - B_aligned, axis=2)

In [4]:
A_aligned.shape

(13, 1, 20)

In [5]:
B_aligned.shape

(1, 11, 20)

In [6]:
distances_2.shape

(13, 11)

In [7]:
np.allclose(distances_1, distances_2)

True

## Option 2: Using the properties of Euclidean distance

By definition, $$ D(\vec{u}, \vec{v}) = \sqrt{\sum_{i=1}^d (\vec{u}_i - \vec{v}_i)^2 } $$

where $d$ is the dimensionality of the vectors. Expanding all summands, we get

$$ D(\vec{u}, \vec{v}) = (
    \vec{u}_1^2 - 2\vec{u}_1\vec{v}_1 + \vec{v}_1^2 + \\
    \vec{u}_2^2 - 2\vec{u}_2\vec{v}_2 + \vec{v}_2^2 + \\
    \dots + \vec{u}_d^2 - 2\vec{u}_d\vec{v}_d + \vec{v}_d^2
)^{1/2}$$

If we group together all the squared elements of $\vec{u}$ and $\vec{v}$ and the stuff in between we see that by definition of the dot product of two vectors is actually equivalent to

$$ D(\vec{u}, \vec{v}) = (\vec{u} \cdot \vec{u} -2 \vec{u} \cdot \vec{v} + \vec{v} \cdot \vec{v})^{1/2} $$

And since matrix multiplication is exactly taking dot products of all pairs of rows or columns of two matrices (depending on how we orient them), we can first compute $-2AB^T$ to get a matrix of size (m, n) and then add sums of squares of rows of A to each row and sums of squares of rows of B to each column to get pairwise distances.

Now we don't need to copy the data to align the rows and only store two additional vectors for sums of squares.

In [ ]:
A_sums = np.sum(np.square(A), axis=1)
B_sums = np.sum(np.square(B), axis=1)
distances_3 = np.sqrt(
    # Elements of A_sums are replicated over columns (each row should contain the same element several times),
    # so we represent it is a single-column matrix.
    # Elements of B_sums are copied over rows, so we represent it as a single-row matrix
    -2 * np.dot(A, B.T) + A_sums.reshape(-1, 1) + B_sums.reshape(1, -1)
)

In [ ]:
np.allclose(distances_1, distances_3)

True